# 🤖 Building a Tool-Calling AI Agent with LangChain & LangGraph
## A Step-by-Step Guide to Understanding How AI Agents Make Decisions and Use Tools

#v📚 What This Notebook Demonstrates
This notebook provides a complete, hands-on walkthrough of building a real AI agent that can autonomously decide when and how to use tools. Unlike simple chatbots that only generate text, this agent can:

## Key Concepts You'll Learn:

* 🧠 Autonomous Decision Making: How AI agents analyze user requests and decide whether they need to use external tools or can answer directly
🔧 Tool Integration: How to give your AI agent "abilities" through custom tools that extend its capabilities beyond just conversation
* 🔄 Workflow Orchestration: How LangGraph creates a decision flow that allows the agent to think, act, observe results, and respond intelligently
* 🏗️ Modular Architecture: How to structure agent systems with clear separation between thinking (LLM), acting (tools), and routing (graph)

## What We Build:
A functional weather assistant agent that:

1. Understands natural language requests like "What's the weather in Boston?"
2. Decides it needs to use the weather tool (not just make up information)
3. Executes the tool with the correct parameters
4. Synthesizes the tool's output into a natural, helpful response

## Why This Matters:
This pattern is the foundation for more complex AI systems that can:

* Search databases and APIs
* Perform calculations and data analysis
* Interact with external systems
* Chain multiple actions together to solve complex problems
* Make intelligent decisions about what tools to use and when

By the end of this notebook, you'll understand not just how to build an AI agent, but why each component exists and how they work together to create truly autonomous AI systems that can take actions on behalf of users.

In [1]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool

# 🤖 Connecting to the Language Model
This code establishes a connection to our AI agent's "brain" - the Large Language Model (LLM) that will process requests and make decisions about when to use tools.

## What's happening here:
```python
llm = ChatOpenAI(
    base_url="http://localhost:8082/v1",  # Points to local vLLM server instead of OpenAI
    api_key="not-needed",                  # vLLM doesn't require authentication
    model="mistralai/Mistral-7B-Instruct-v0.3"  # The actual model running on vLLM
)
```

## Key Points:

* ChatOpenAI class: Even though we're using Mistral (not OpenAI), LangChain's ChatOpenAI class works because vLLM provides an OpenAI-compatible API endpoint. This is a common pattern - many LLM servers mimic OpenAI's API format for compatibility.
* base_url: Instead of connecting to OpenAI's servers, we're pointing to localhost:8082 where vLLM is running locally. This means the model is running on your own machine, giving you full control and privacy.
* api_key="not-needed": Unlike OpenAI's API which requires authentication, our local vLLM server doesn't need an API key. We still have to provide this parameter because the ChatOpenAI class expects it.
* model: Specifies Mistral-7B-Instruct v0.3, a powerful open-source model that's been fine-tuned to follow instructions and engage in helpful dialogue. This model will power our agent's reasoning and decision-making.

## 💡 Think of this as: Creating a direct phone line to an AI assistant that lives on your computer rather than in the cloud. The agent will use this connection to think through problems and decide when to use tools.

In [2]:
# Connect to vLLM
llm = ChatOpenAI(
    base_url="http://localhost:8082/v1",
    api_key="not-needed",
    model="mistralai/Mistral-7B-Instruct-v0.3"
)

# 🛠️ Creating a Tool for the Agent
This code defines a tool that our AI agent can choose to use when it needs weather information. Think of tools as special abilities or functions the agent can call upon when needed.

## What's happening here:
```python
@tool
def get_weather(location: str) -> str:
    """
    Retrieves current weather information for a specified location.
    
    Args:
        location: The city name and optionally state/country (e.g., "San Francisco, CA" or "London, UK")
    
    Returns:
        A string containing the current temperature in Fahrenheit and weather conditions.
    """
    return f"Current weather in {location}: Temperature is 72°F, conditions are sunny with clear skies"
```

## Key Components:

* @tool decorator: This special decorator from LangChain transforms a regular Python function into a tool that the AI agent can discover and use. It's like registering this function in the agent's toolbox.
* Type hints (location: str -> str): These tell the agent exactly what type of input the tool expects and what it returns. This helps the LLM make correct tool calls.
* The docstring is CRITICAL:

    * The agent reads this description to understand when and how to use the tool
    * Without a clear docstring, the agent might not know this tool exists or how to use it properly
    * The docstring acts as an "instruction manual" for the AI


* Mock implementation: Currently returns hardcoded weather data (always 72°F and sunny). In a production system, this would make an actual API call to a weather service.

## How the Agent Uses This Tool:

1. User asks: "What's the weather in Boston?"
2. Agent reads the tool's docstring and thinks: "This tool can get weather for a location"
3. Agent decides: "I need to use the get_weather tool with 'Boston' as the location"
4. Agent calls: get_weather("Boston")
5. Agent receives: The weather data and formats it into a natural response

## 💡 Important: The quality of your docstring directly impacts how well the agent can use your tool. Always write clear, detailed descriptions that explain what the tool does and what parameters it needs.

In [3]:
# Define a tool with proper docstring
@tool
def get_weather(location: str) -> str:
    """
    Retrieves current weather information for a specified location.
    
    Args:
        location: The city name and optionally state/country (e.g., "San Francisco, CA" or "London, UK")
    
    Returns:
        A string containing the current temperature in Fahrenheit and weather conditions.
    """
    return f"Current weather in {location}: Temperature is 72°F, conditions are sunny with clear skies"


# 🔗 Connecting Tools to the Language Model
This code gives our AI agent access to the tools we've created, enabling it to discover and use them during conversations.

## What's happening here:
```python
tools = [get_weather]
llm_with_tools = llm.bind_tools(tools)
```

## Breaking it down:

* tools = [get_weather]: Creates a list of all available tools. In this case, we only have one tool (get_weather), but you could add more:

```python 
tools = [get_weather, search_web, send_email, calculate_math]
```

* llm.bind_tools(tools): This is where the magic happens! The bind_tools() method:

    * Tells the LLM what tools are available
    * Automatically extracts tool descriptions from the docstrings
    * Configures the model to output specially formatted tool calls when needed
    * Returns a new LLM instance that's "tool-aware"



## What changes after binding:
Before binding (llm):

* Can only respond with text
* Doesn't know about any tools
* Would just say "I can't check the weather"

## After binding (llm_with_tools):

* Knows about the get_weather tool
* Can decide when to use it
* Can format proper tool calls with the right parameters
* Can chain tool usage with responses

## 💡 Think of it like this: Binding tools is like giving someone a Swiss Army knife and teaching them what each tool does. Now when they encounter a problem, they can choose the right tool for the job instead of just talking about it.
## Under the hood:
When you bind tools, LangChain actually modifies the system prompt to include information about available tools and how to call them. The LLM will now respond with special structured outputs when it wants to use a tool, which LangGraph can intercept and execute.

In [4]:
# Bind tools
tools = [get_weather]
llm_with_tools = llm.bind_tools(tools)

# 🎯 The Agent's Decision-Making Function
This function is the core of our agent - it's where the AI receives messages, thinks about them, and decides whether to respond directly or use a tool.

## What's happening here:
```python
def call_model(state: MessagesState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}
```

## Breaking down each part:

* state: MessagesState: The current conversation state

    * Contains the full message history (user messages, AI responses, tool results)
    * LangGraph automatically maintains and passes this state between nodes
    * Think of it as the agent's "memory" of the conversation


* llm_with_tools.invoke(state["messages"]): This is where the thinking happens

    * Sends the entire conversation history to the LLM
    * The LLM analyzes the latest message and decides what to do
    * Returns either:

        * A regular text response, or
        * A special tool-call request (e.g., "I need to use get_weather for Boston")




* return {"messages": [response]}: Updates the conversation state

    * Wraps the response in a list and dictionary format that LangGraph expects
    * Adds the new response to the conversation history
    * This updated state flows to the next node in the graph



## How it works in practice:

1. User says: "What's the weather in Boston?"
2. Function receives: All previous messages plus this new one
3. LLM analyzes: "Hmm, they want weather info, I have a weather tool..."
4. Function returns: Either a tool call or direct response
5. LangGraph routes: Based on the response type, goes to tools node or ends

## 💡 Key insight: 
This function doesn't actually execute tools - it just decides whether tools are needed. The actual tool execution happens in the **ToolNode** that we'll see next. This separation of concerns makes the agent modular and easy to extend.

In [5]:
def call_model(state: MessagesState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}


# 🕸️ Building the Agent's Workflow Graph
This code constructs the "brain map" of our agent - defining how it flows between thinking, using tools, and responding. LangGraph uses a graph structure where nodes are functions and edges are the paths between them.

## What's happening here:
```python 
graph_builder = StateGraph(MessagesState)
graph_builder.add_node("agent", call_model)
graph_builder.add_node("tools", ToolNode(tools))
graph_builder.add_conditional_edges(
    "agent",
    lambda x: "tools" if x["messages"][-1].tool_calls else "__end__"
)
graph_builder.add_edge("tools", "agent")
graph_builder.set_entry_point("agent")
graph = graph_builder.compile()
```

**Step-by-step breakdown:**

1. **`StateGraph(MessagesState)`**: Creates a new graph that tracks conversation messages
   - Think of this as drawing an empty flowchart
   - `MessagesState` defines what kind of data flows through the graph

2. **`add_node("agent", call_model)`**: Adds the agent's thinking node
   - Names it "agent" 
   - Links it to our `call_model` function
   - This is where the LLM makes decisions

3. **`add_node("tools", ToolNode(tools))`**: Adds the tool execution node
   - `ToolNode` is a pre-built component that knows how to execute tools
   - It receives tool calls from the agent and runs the actual functions

4. **`add_conditional_edges(...)`**: Creates smart routing from the agent node
   - Checks if the last message contains tool calls
   - If yes → routes to "tools" node
   - If no → routes to "__end__" (conversation complete)
   - This is the decision point: "Should I use a tool or just respond?"

5. **`add_edge("tools", "agent")`**: Always return to agent after using a tool
   - After executing a tool, go back to the agent
   - The agent can then use the tool's output to formulate a response

6. **`set_entry_point("agent")`**: Defines where conversations start
   - Every new message first goes to the agent node

7. **`graph.compile()`**: Finalizes and optimizes the graph for execution

### Visual representation of the flow:
```
User Message
     ↓
  [AGENT] ← (starts here)
     ↓
  Decides: Need tool?
   ↙        ↘
 Yes         No
  ↓           ↓
[TOOLS]    [END]
  ↓
Back to AGENT
  ↓
[Response to User]


💡 The beauty of this design: The agent can chain multiple tool calls if needed. For example, if asked "What's the weather in Boston and NYC?", it could:

Call weather tool for Boston
Return to agent
Call weather tool for NYC
Return to agent
Synthesize both results into a final response

This creates a flexible, iterative problem-solving loop rather than a rigid linear flow.

In [6]:
# Build graph
graph_builder = StateGraph(MessagesState)
graph_builder.add_node("agent", call_model)
graph_builder.add_node("tools", ToolNode(tools))

graph_builder.add_conditional_edges(
    "agent",
    lambda x: "tools" if x["messages"][-1].tool_calls else "__end__"
)
graph_builder.add_edge("tools", "agent")
graph_builder.set_entry_point("agent")

graph = graph_builder.compile()

# 🧪 Testing the Agent's Tool-Calling Ability
This code demonstrates how to use our agent and verify that it's actually using tools rather than just generating responses.

## What's happening here:
```python
# Test the agent's tool calling
result = graph.invoke({
    "messages": [("user", "What's the weather in Boston?")]
})
print(result["messages"][-1].content)

# Verify that tool call actually occurred
for msg in result["messages"]:
    print(f"\nType: {type(msg).__name__}")
    print(f"Content: {msg.content}")
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"Tool calls: {msg.tool_calls}")
```

## Breaking it down:
## Part 1: Invoking the Agent

* graph.invoke(...): Starts the agent workflow

    * Input format: Dictionary with "messages" key
    * Message format: Tuple of ("user", "question")
    * This kicks off the entire agent→tools→agent flow


* result["messages"][-1].content: Gets the final response

    * The result contains ALL messages from the conversation
    * [-1] gets the last message (the agent's final response)
    * This is what the user would actually see



## Part 2: Debugging & Verification
The second loop shows us the complete conversation flow:
```python
for msg in result["messages"]:
    print(f"\nType: {type(msg).__name__}")
    print(f"Content: {msg.content}")
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"Tool calls: {msg.tool_calls}")
```

This reveals the behind-the-scenes process:

1. **HumanMessage**: "What's the weather in Boston?"
2. **AIMessage**: Contains tool_calls for get_weather
3. **ToolMessage**: The actual weather data returned
4. **AIMessage**: Final natural language response

### Expected Output:
```
The weather in Boston is currently 72°F and sunny with clear skies!

Type: HumanMessage
Content: What's the weather in Boston?

Type: AIMessage
Content: 
Tool calls: [{'name': 'get_weather', 'args': {'location': 'Boston'}}]

Type: ToolMessage
Content: Current weather in Boston: Temperature is 72°F, conditions are sunny with clear skies

Type: AIMessage
Content: The weather in Boston is currently 72°F and sunny with clear skies!
```

## 💡 Why this verification matters:

Confirms tool usage: We can see the agent actually called the tool rather than hallucinating weather data
Debugging aid: If something goes wrong, we can see exactly where in the flow it failed

In [7]:
# Test the agent's tool calling
result = graph.invoke({
    "messages": [("user", "What's the weather in Boston?")]
})

print(result["messages"][-1].content)

# verify that tool call actually occurred
for msg in result["messages"]:
    print(f"\nType: {type(msg).__name__}")
    print(f"Content: {msg.content}")
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"Tool calls: {msg.tool_calls}")

 The weather in Boston is currently sunny with clear skies and the temperature is 72°F. Enjoy your day!

Type: HumanMessage
Content: What's the weather in Boston?

Type: AIMessage
Content: 
Tool calls: [{'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'yriRmzvtR', 'type': 'tool_call'}]

Type: ToolMessage
Content: Current weather in Boston: Temperature is 72°F, conditions are sunny with clear skies

Type: AIMessage
Content:  The weather in Boston is currently sunny with clear skies and the temperature is 72°F. Enjoy your day!


# 🚀 Next Steps: From Mock to Production
Now that you understand the fundamentals of tool-calling agents, here's how to evolve this into a production-ready system:

## Immediate Enhancements:

1. Replace Mock Tools with Real APIs

    * Integrate actual weather APIs (OpenWeatherMap, Weather.gov)
    * Add error handling for API failures and rate limits
    * Implement retry logic and fallback strategies


2. Expand the Toolset

    * Add complementary tools (forecast, weather alerts, historical weather)
    * Create tools for different domains (news, calculations, web search)
    * Build tool chains where one tool's output feeds into another


3. Enhance Agent Intelligence

    * Add memory/context persistence across conversations
    * Implement planning capabilities for multi-step problems
    * Fine-tune prompts for better tool selection decisions

## Advanced Patterns:

* Tool Validation: Add parameter validation before tool execution
* Parallel Execution: Configure the agent to call multiple tools simultaneously
* Observability: Add logging and monitoring to track tool usage and performance
* Human-in-the-Loop: Build approval workflows for sensitive tool operations